In [1]:
import numpy as np
import random
import time
import matplotlib.pyplot as plt
import copy as cp
import pickle
import pandas as pd

import StructureMatrixGenerator as smg
import trivialSimpleUpdate as tsu
import DoubleEdgeFactorGraphs as defg
import SimpleUpdate as su
import bmpslib as bmps

# Comparing trivial-SImple Update (tSU) and Belief Propagation (BP) fixed-point convergence on random PEPS

**=-=-=-=-=-=-=-=-=-=  SUMMARY  =-=-=-=-=-=-=-=-=-=**

In this experiment I inizilize 4 copies of a randomaly drawn **Projected Entangled Pair State (PEPS)** and compared the fixed-point two-body **reduced density matrices (RDMs)** of these copies using 4 different algorithms:
1. Belief Propagation (BP)
2. trivial-SU (parallel scheduling)
3. trivial-SU (serial scheduling)
2. Boundary Matrix Product Operator (BMPO)

The differerence between the serial and parallel tSU scheduling is as follows:
Parallel: iterating over all of the tensor network edges in some fixed order and update every two tensors using a local SVD operation. The updated tensors than put back into the tensor network and we move to the next edge in order.
Series:  iterating over all of the tensor network edges in some fixed order and update every two tensors using a local SVD operation. The two updated tensors are held in a copy tensor network and are not used in the rest of the update scheme. Thus, making a clear separation between the tensor network at time $t$ to the updated tensor network at time $t+1$.

**=-=-=-=-=-=-=-=-=  CONCLUSIONS  =-=-=-=-=-=-=-=-=**

In retrospective it turns out that the **trivial-SU (serial scheduling)** is nonsense. If a tensor has more then one virtual edge it is not clear which tensor would take its place at the next iteration because it would be updated according to the number of its entering edges. Numerically, these scheduling scheme is taking much more time to converge to a fixed-point and from the numerical result in this notebook it seems that this fixed-point is different from the one shared by BP and tSU "parallel".

Finally, I kept the code as is, with all 4 algorithms. 

In [2]:
# tSU and BP parameters
N, M = 8, 8                                                   # NxM PEPS
bc = 'open'                                                   # boundary conditions
dw = 1e-6                                                     # maximal error allowed between two-body RDMS
d = 2                                                         # tensor network physical bond dimension
D_max = 3                                                     # maximal virtual bond dimension allowed for truncation
t_max = 80                                                    # maximal number of BP iterations
epsilon = 1e-10                                               # convergence criteria for BP messages (not used)
dumping = 0.                                                  # BP messages dumping between [0, 1]
iterations = 100                                              # maximal number of tSU iterations
smat, _ = smg.finitePEPSobcStructureMatrixGenerator(N, M)     # generating the PEPS structure matrix
n, m = smat.shape

### Initializing random PEPS copies

In [3]:
# get random PEPS tensor network
tensors, weights = smg.randomTensornetGenerator(smat, d, D_max)

# get copies of tensor network
BP_tensors, BP_weights           = cp.deepcopy(tensors), cp.deepcopy(weights)
BMPO_tensors, BMPO_weights       = cp.deepcopy(tensors), cp.deepcopy(weights)
tsu_par_tensors, tsu_par_weights = cp.deepcopy(tensors), cp.deepcopy(weights)
tsu_ser_tensors, tsu_ser_weights = cp.deepcopy(tensors), cp.deepcopy(weights)

### BP:

In [4]:
# constructing the dual double-edge factor graph and run a single BP iteration
graph = defg.defg()
graph = su.TNtoDEFGtransform(graph, BP_tensors, BP_weights, smat)
graph.sumProduct(1, epsilon, dumping, initializeMessages=1, printTime=1, RDMconvergence=0)
BP_rdm = []
for j in range(m):
        BP_rdm.append(tsu.BPdoubleSiteRDM1(j, BP_tensors, BP_weights, smat, cp.deepcopy(graph.messages_n2f)))

In [5]:
 # run BP and calculate two body rdms between two consecutive BP iterations
for t in range(t_max):
    graph.sumProduct(1, epsilon, dumping, initializeMessages=1, printTime=1, RDMconvergence=0)
    
    ATD_BP = 0
    BP_rdm_next = []
    for j in range(m):
        BP_rdm_next.append(tsu.BPdoubleSiteRDM1(j,
                                                BP_tensors,
                                                BP_weights,
                                                smat,
                                                cp.deepcopy(graph.messages_n2f)))
        
        ATD_BP += tsu.traceDistance(BP_rdm_next[j], BP_rdm[j])
        BP_rdm[j] = BP_rdm_next[j]
    ATD_BP /= m
    print('The ATD_BP is: {} at iteration {}'.format(ATD_BP, t))
    if ATD_BP < dw:
        print('\n')
        print('The final ATD_BP is: {} at iteration {}'.format(ATD_BP, t + 2))
        break

The ATD_BP is: 0.0011071729536162873 at iteration 0
The ATD_BP is: 0.00016521335137383646 at iteration 1
The ATD_BP is: 2.255061035485507e-05 at iteration 2
The ATD_BP is: 3.412911591428528e-06 at iteration 3
The ATD_BP is: 4.765417199678164e-07 at iteration 4


The final ATD_BP is: 4.765417199678164e-07 at iteration 6


### tSU "parallel" scheduling:

In [6]:
# calculate the double site rdm in parallel tsu
tSU_par_rdm = []
for i in range(m):
    tSU_par_rdm.append(tsu.doubleSiteRDM(i, tsu_par_tensors, tsu_par_weights, smat))

In [7]:
# parallel trivial SU run
for i in range(iterations):
    tsu_par_tensors_next, tsu_par_weights_next = tsu.trivialsimpleUpdate(tsu_par_tensors,
                                                                         tsu_par_weights,
                                                                         smat,
                                                                         D_max, 
                                                                         scheduling='parallel')
    ATD = 0
    tSU_par_rdm_next = []
    for j in range(m):
        tSU_par_rdm_next.append(tsu.doubleSiteRDM(j, tsu_par_tensors_next, tsu_par_weights_next, smat))
        ATD += tsu.traceDistance(tSU_par_rdm_next[j], tSU_par_rdm[j])
        tSU_par_rdm[j] = tSU_par_rdm_next[j]
    ATD /= m
    if ATD < dw:
        print('The ATD is: {} at iteration {}'.format(ATD, i))
        tsu_par_tensors = tsu_par_tensors_next
        tsu_par_weights = tsu_par_weights_next
        break
    tsu_par_tensors = tsu_par_tensors_next
    tsu_par_weights = tsu_par_weights_next

/Users/royelkabetz/Git/QBPmodule/trivialSimpleUpdate.py:336: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  leftDim = np.prod(shape[[leftIdx]])
/Users/royelkabetz/Git/QBPmodule/trivialSimpleUpdate.py:337: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  rightDim = np.prod(shape[[rightIdx]])


The ATD is: 2.3217774106293488e-07 at iteration 7


### tSU "series" scheduling:

In [8]:
# calculate the double site rdm in series tsu
tSU_ser_rdm = []
for i in range(m):
    tSU_ser_rdm.append(tsu.doubleSiteRDM(i, tsu_ser_tensors, tsu_ser_weights, smat))

In [9]:
# series trivial SU run
for i in range(iterations):
    tsu_ser_tensors_next, tsu_ser_weights_next = tsu.trivialsimpleUpdate(tsu_ser_tensors,
                                                                         tsu_ser_weights,
                                                                         smat,
                                                                         D_max, 
                                                                         scheduling='series')
    ATD = 0
    tSU_ser_rdm_next = []
    for j in range(m):
        tSU_ser_rdm_next.append(tsu.doubleSiteRDM(j, tsu_ser_tensors_next, tsu_ser_weights_next, smat))
        ATD += tsu.traceDistance(tSU_ser_rdm_next[j], tSU_ser_rdm[j])
        tSU_ser_rdm[j] = tSU_ser_rdm_next[j]
    ATD /= m
    if ATD < dw:
        print('The ATD is: {} at iteration {}'.format(ATD, i))
        tsu_ser_tensors = tsu_ser_tensors_next
        tsu_ser_weights = tsu_ser_weights_next
        break
    tsu_ser_tensors = tsu_ser_tensors_next
    tsu_ser_weights = tsu_ser_weights_next

The ATD is: 9.958176849308073e-07 at iteration 17


### Exact calculations using BMPO 

In [10]:
# RDMS using BMPO from bmpslib
tensors_p = su.absorbAllTensorNetWeights(cp.deepcopy(BMPO_tensors), cp.deepcopy(BMPO_weights), smat)
tensors_p = smg.PEPS_OBC_broadcast_to_Itai(tensors_p, [N, M], d, D_max)
peps = bmps.peps(N, M)
for t, T in enumerate(tensors_p):
    i, j = np.unravel_index(t, [N, M])
    peps.set_site(T, i, j)
bmpo_RDMS = bmps.calculate_PEPS_2RDM(peps, int(2 * (D_max ** 2)))
for i in range(len(bmpo_RDMS)):
    bmpo_RDMS[i] = np.reshape(bmpo_RDMS[i], (d * d, d * d))
print('edge 0 rdm: \nreal part\n{}\nimaginary part\n{}'.format(np.real(bmpo_RDMS[0]), np.imag(bmpo_RDMS[0])))


edge 0 rdm: 
real part
[[0.22821273 0.24926758 0.24926758 0.27386604]
 [0.2289367  0.24244176 0.25222962 0.26814698]
 [0.2289367  0.25222962 0.24244176 0.26814698]
 [0.23309635 0.24812212 0.24812212 0.26482488]]
imaginary part
[[-3.83798872e-19  1.77378517e-02 -1.77378517e-02 -5.58011374e-18]
 [ 2.79215577e-02  4.00802687e-02  1.26869799e-02  2.49456363e-02]
 [-2.79215577e-02 -1.26869799e-02 -4.00802687e-02 -2.49456363e-02]
 [ 3.54740977e-18  1.05610886e-02 -1.05610886e-02  2.41650284e-18]]


In [11]:
def getBMPOedgeList(N, M, smat):
    TN = np.arange(N * M).reshape(N, M)
    Hpairs = []
    Vpairs = []
    HedgeList = []
    VedgeList = []
    for i in range(N):
        for j in range(M - 1):
            tH1 = TN[i][j]
            tH2 = TN[i][j + 1]
            Hpairs.append([tH1, tH2])

    for i in range(N - 1):
        for j in range(M):
            tV1 = TN[i][j]
            tV2 = TN[i + 1][j]
            Vpairs.append([tV1, tV2])

    for i, pair in enumerate(Hpairs):
        for k in range(m):
            if smat[pair[0], k] and smat[pair[1], k]:
                HedgeList.append(k)
                break
    for i, pair in enumerate(Vpairs):
        for k in range(m):
            if smat[pair[0], k] and smat[pair[1], k]:
                VedgeList.append(k)
                break

    return HedgeList + VedgeList

### Comparing two-body reduced density matrices (RDM) between BP, tSU "parallel", tSU "series", BMPO 

In [12]:
# get the order of the BMPO edges and calculate the ATDs
edgeList = getBMPOedgeList(N, M, smat)

# ATD
ATD_bmpo_BP = 0
ATD_bmpo_tsu_par = 0
ATD_bmpo_tsu_ser = 0
ATD_BP_tsu_par = 0
ATD_BP_tsu_ser = 0
ATD_tsu_par_ser = 0

for i in range(m):
    ATD_bmpo_BP += tsu.traceDistance(bmpo_RDMS[i], BP_rdm[edgeList[i]])
    ATD_bmpo_tsu_par += tsu.traceDistance(bmpo_RDMS[i], tSU_par_rdm[edgeList[i]])
    ATD_bmpo_tsu_ser += tsu.traceDistance(bmpo_RDMS[i], tSU_ser_rdm[edgeList[i]])
    ATD_BP_tsu_par += tsu.traceDistance(BP_rdm[edgeList[i]], tSU_par_rdm[edgeList[i]])
    ATD_BP_tsu_ser += tsu.traceDistance(BP_rdm[edgeList[i]], tSU_ser_rdm[edgeList[i]])
    ATD_tsu_par_ser += tsu.traceDistance(tSU_ser_rdm[edgeList[i]], tSU_par_rdm[edgeList[i]])
    
ATD_bmpo_BP /= m
ATD_bmpo_tsu_par /= m
ATD_bmpo_tsu_ser /= m
ATD_BP_tsu_par /= m
ATD_BP_tsu_ser /= m
ATD_tsu_par_ser /= m

In [13]:
# print results to screen
print('\n=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-')
print('|    {}x{} random PEPS tensor network (d,D) = ({},{})    |'.format(N, M, d, D_max))
print('|<><><><><><><><><><><><><><><><><><><><><><><><><><>|')
print('|     Averaged Trace Distance (ATD) calculations     |'  )
print('=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-'  )
print('| RESULTS:                                          '  )
print('|                                                   '  )
print('| ATD(BMPO - BP)                       = {}     '.format(np.round(ATD_bmpo_BP, 8)))
print('| ATD(BMPO - tSU "parallel")           = {}     '.format(np.round(ATD_bmpo_tsu_par, 8)))
print('| ATD(BMPO - tSU "series")             = {}     '.format(np.round(ATD_bmpo_tsu_ser, 8)))
print('| ATD(BP - tSU "parallel")             = {}          '.format(np.round(ATD_BP_tsu_par, 8)))
print('| ATD(BP - tSU "series")               = {}     '.format(np.round(ATD_BP_tsu_ser, 8)))
print('| ATD(tSU "series" - "parallel")       = {}     '.format(np.round(ATD_tsu_par_ser, 8)))
print('|                                                   '  )
print('=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-'  )



=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-
|    8x8 random PEPS tensor network (d,D) = (2,3)    |
|<><><><><><><><><><><><><><><><><><><><><><><><><><>|
|     Averaged Trace Distance (ATD) calculations     |
=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-
| RESULTS:                                          
|                                                   
| ATD(BMPO - BP)                       = 0.03891582     
| ATD(BMPO - tSU "parallel")           = 0.03891583     
| ATD(BMPO - tSU "series")             = 0.24034837     
| ATD(BP - tSU "parallel")             = 7e-08          
| ATD(BP - tSU "series")               = 0.24046608     
| ATD(tSU "series" - "parallel")       = 0.24046608     
|                                                   
=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-
